In [1]:
import csv
from gensim.models.word2vec import Word2Vec
import pandas as pd
import numpy as np
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer


In [2]:
from collections import Counter

# train은 업샘플링 한거, test은 안한 채로 만든 데이터 셋 불러옴

In [3]:
pre_train = pd.read_csv('final_train.csv')
pre_test = pd.read_csv('final_test.csv')

In [4]:
pre_train

,Unnamed: 0,M_SAD,SitTime,PA_TOT,I_SB_EX,I_SB_FR,M_SUI_CON,i_SB_BK,BMI,M_STR,F_SODA,PA_MSC
0,0,M_SAD_1,SitTime_4,PA_TOT_1,I_SB_EX_3,I_SB_FR_3,M_SUI_CON_1,i_SB_BK_3,BMI_1,M_STR_3,F_SODA_3,PA_MSC_1
1,1,M_SAD_1,SitTime_4,PA_TOT_6,I_SB_EX_2,I_SB_FR_4,M_SUI_CON_1,i_SB_BK_5,BMI_2,M_STR_3,F_SODA_2,PA_MSC_6
2,2,M_SAD_1,SitTime_2,PA_TOT_1,I_SB_EX_5,I_SB_FR_3,M_SUI_CON_1,i_SB_BK_5,BMI_1,M_STR_3,F_SODA_2,PA_MSC_2
3,3,M_SAD_1,SitTime_4,PA_TOT_7,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_1,i_SB_BK_2,BMI_1,M_STR_4,F_SODA_4,PA_MSC_3
4,4,M_SAD_1,SitTime_5,PA_TOT_2,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_1,i_SB_BK_2,BMI_2,M_STR_3,F_SODA_2,PA_MSC_6
...,...,...,...,...,...,...,...,...,...,...,...,...
59347,59347,M_SAD_2,SitTime_3,PA_TOT_1,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_2,i_SB_BK_4,BMI_1,M_STR_1,F_SODA_2,PA_MSC_1
59348,59348,M_SAD_2,SitTime_5,PA_TOT_5,I_SB_EX_4,I_SB_FR_3,M_SUI_CON_2,i_SB_BK_4,BMI_1,M_STR_1,F_SODA_3,PA_MSC_4
59349,59349,M_SAD_2,SitTime_2,PA_TOT_1,I_SB_EX_1,I_SB_FR_2,M_SUI_CON_2,i_SB_BK_5,BMI_1,M_STR_1,F_SODA_1,PA_MSC_1
59350,59350,M_SAD_1,SitTime_2,PA_TOT_1,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_2,i_SB_BK_2,BMI_2,M_STR_5,F_SODA_1,PA_MSC_1


In [4]:
pre_train = pre_train.drop(columns = ['Unnamed: 0'])
pre_test = pre_test.drop(columns = ['Unnamed: 0'])

<b>Train shuffle해줌.

In [5]:
pre_train=pre_train.sample(frac=1).reset_index(drop=True)

In [6]:
pre_train.shape

(59352, 11)

In [34]:
#pre_train.to_csv('good_pretrain.csv')

# cbow모델에 넣기 좋게 리스트 형식으로 변환

In [7]:
train_list = pre_train.values.tolist()
test_list = pre_test.values.tolist()

In [9]:
len(train_list)

59352

<b> Tokenizing

In [8]:
def tokenize(corpus):
    """
    Tokenize the corpus of text.
    :param corpus: list containing a string of text (example: ["I like playing football with my friends"])
    :return corpus_tokenized: indexed list of words in the corpus, in the same order as the original corpus (the example above would return [[1, 2, 3, 4]])
    :return V: size of vocabulary
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    corpus_tokenized = tokenizer.texts_to_sequences(corpus)
    V = len(tokenizer.word_index)
    return corpus_tokenized, V

In [9]:
cor, V = tokenize(train_list)
V

58

In [10]:
cor_te, V_te = tokenize(test_list)
V_te

58

# 모델 훈련

In [11]:
model_cbow = Word2Vec(train_list, workers = 6, size = 58 , min_count = 1, window = 5, sg = 0)

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score

## 실제 라벨 값을 저장함. actual_tr  : train의 자살 라벨값, acutal_te : test의 자살 라벨값

In [15]:
actual_tr = []
for i in range(len(train_list)):
    actual_tr.append(train_list[i][5])

actual_te = []
for i in range(len(test_list)):
    actual_te.append(test_list[i][5])

In [37]:
# Performance Measure 출력 함수 만들기
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    F2_Score = (5 * Recall * Precision) / (Recall + 4*Precision) # Recall을 Precision보다 2배 중요하게 생각하여 F2 Score 사용
    
    print("Accuracy: ", Accuracy) 
    print("Precision: ", Precision)
    print("Recall: ", Recall)
    print("Specificity: ", Specificity)
    print("F1_Score: ", F1_Score)
    print("F2_Score: ", F2_Score)

## 구리게 나온 척도 : 비슷한 코사인 유사도를 가진 것으로 예측

*** prediction based on counting similarity ***

In [16]:
def predict_cbow_count(test_list):
    predict_count_list = []
    for i in range(len(test_list)):
        count_CON2 = 0
        count_CON1 = 0
        for j in range(len(test_list[i])):
            if j == 5:
                continue
            elif model_cbow.wv.similarity('M_SUI_CON_2', test_list[i][j]) > model_cbow.wv.similarity('M_SUI_CON_1', test_list[i][j]):
                count_CON2 += 1
            else:
                count_CON1 += 1
        if count_CON2 > count_CON1:
            predict_count_list.append('M_SUI_CON_2')
        else:
            predict_count_list.append('M_SUI_CON_1')
    return predict_count_list

In [17]:
#훈련시킨 것 예측 값
predict_cbow_count_train = predict_cbow_count(train_list)
predict_cbow_count_test = predict_cbow_count(test_list)

예측한 결과 분포 확인

In [18]:
Counter(predict_cbow_count_train)

Counter({'M_SUI_CON_1': 32893, 'M_SUI_CON_2': 26459})

In [19]:
Counter(predict_cbow_count_test)

Counter({'M_SUI_CON_1': 10131, 'M_SUI_CON_2': 4402})

In [20]:
results_count_tr = confusion_matrix(actual_tr, predict_cbow_count_train)
print(results_count_tr)
print(accuracy_score(actual_tr, predict_cbow_count_train))
print(classification_report(actual_tr, predict_cbow_count_train))

[[21855  7821]
 [11038 18638]]
0.6822516511659253
              precision    recall  f1-score   support

 M_SUI_CON_1       0.66      0.74      0.70     29676
 M_SUI_CON_2       0.70      0.63      0.66     29676

    accuracy                           0.68     59352
   macro avg       0.68      0.68      0.68     59352
weighted avg       0.68      0.68      0.68     59352



In [21]:
results_count_te = confusion_matrix(actual_te, predict_cbow_count_test)
print(results_count_te)
print(accuracy_score(actual_te, predict_cbow_count_test))
print(classification_report(actual_te, predict_cbow_count_test))

[[9340 3334]
 [ 791 1068]]
0.7161632147526319
              precision    recall  f1-score   support

 M_SUI_CON_1       0.92      0.74      0.82     12674
 M_SUI_CON_2       0.24      0.57      0.34      1859

    accuracy                           0.72     14533
   macro avg       0.58      0.66      0.58     14533
weighted avg       0.84      0.72      0.76     14533



In [ ]:
model_evaluation(actual_te, predict_cbow_count_test)

## 잘 나온 측도(맞나?) 

<b> 각각의 cell의 값들과 라벨 값 사이의 유사도를 측정하여 그 평균값이 더 큰쪽으로 예측하게끔 설정

In [22]:
def predict_cbow_avg_sim(test_list):
    predict_list = []
    for i in range(len(test_list)):
        pos_similarity_avg = 0
        neg_similarity_avg = 0
        for j in range(len(test_list[i])):
            if j == 5:
                continue
            else:
                pos_similarity_avg += model_cbow.wv.similarity('M_SUI_CON_2', test_list[i][j])
                neg_similarity_avg += model_cbow.wv.similarity('M_SUI_CON_1', test_list[i][j])
        if (pos_similarity_avg / (len(test_list[i]) - 1)) >= (neg_similarity_avg / (len(test_list[i]) - 1)):
            predict_list.append('M_SUI_CON_2')
        else:
            predict_list.append('M_SUI_CON_1')
    return predict_list


In [23]:
predict_cbow_sim_train = predict_cbow_avg_sim(train_list)
predict_cbow_sim_test = predict_cbow_avg_sim(test_list)

In [24]:
Counter(predict_cbow_sim_train)

Counter({'M_SUI_CON_2': 31222, 'M_SUI_CON_1': 28130})

In [25]:
Counter(predict_cbow_sim_test)

Counter({'M_SUI_CON_2': 2606, 'M_SUI_CON_1': 11927})

In [26]:
results_tr = confusion_matrix(actual_tr, predict_cbow_sim_train)
print(results_tr)
print(accuracy_score(actual_tr, predict_cbow_sim_train))
print(classification_report(actual_tr, predict_cbow_sim_train))

[[27819  1857]
 [  311 29365]]
0.9634721660601159
              precision    recall  f1-score   support

 M_SUI_CON_1       0.99      0.94      0.96     29676
 M_SUI_CON_2       0.94      0.99      0.96     29676

    accuracy                           0.96     59352
   macro avg       0.96      0.96      0.96     59352
weighted avg       0.96      0.96      0.96     59352



In [27]:
results_te = confusion_matrix(actual_te, predict_cbow_sim_test)
print(results_te)
print(accuracy_score(actual_te, predict_cbow_sim_test))
print(classification_report(actual_te, predict_cbow_sim_test))

[[11893   781]
 [   34  1825]]
0.9439207321268837
              precision    recall  f1-score   support

 M_SUI_CON_1       1.00      0.94      0.97     12674
 M_SUI_CON_2       0.70      0.98      0.82      1859

    accuracy                           0.94     14533
   macro avg       0.85      0.96      0.89     14533
weighted avg       0.96      0.94      0.95     14533



In [39]:
model_evaluation(actual_te, predict_cbow_sim_test)

Accuracy:  0.9439207321268837
Precision:  0.7003069838833461
Recall:  0.9817105970952125
Specificity:  0.9383777812845195
F1_Score:  0.8174692049272115
F2_Score:  0.9086835291774547


In [28]:
pd.Series(actual_te).value_counts()

M_SUI_CON_1    12674
M_SUI_CON_2     1859
dtype: int64

In [29]:
pd.Series(predict_cbow_sim_test).value_counts()

M_SUI_CON_1    11927
M_SUI_CON_2     2606
dtype: int64

In [30]:
from sklearn.metrics import roc_auc_score

<b> aucroc 구할라면 라벨값들이 숫자여야해서 M_SUI_CON_1을 0, M_SUI_CON_2을 1로 바꿔줌 근데 생각해보니 밑에 str로 바뀐 숫자가 들어갔을텐데 걍 계산되네 => roc_auc_score안에서 알아서 바꿔주네

In [31]:
for_auc_te = []
for item in actual_te:
    item_mod = int(item.replace("M_SUI_CON_2" , "1").replace('M_SUI_CON_1' , "0"))
    for_auc_te.append(item_mod)

In [32]:
for_auc_pred = []
for item in predict_cbow_sim_test:
    item_mod = int(item.replace("M_SUI_CON_2" , "1").replace('M_SUI_CON_1' , "0"))
    for_auc_pred.append(item_mod)

In [33]:
roc_auc_score(for_auc_te, for_auc_pred)

0.960044189189866